In [25]:
import pandas as pd
import time
import datetime
from Covered_calls.py import get_processed_dataframe


SyntaxError: invalid syntax (4101332585.py, line 4)

In [15]:
# Define a function to check if a dataframe has been fetched for the current date
def is_dataframe_fetched_today(date_dataframes):
    today = time.strftime('%Y-%m-%d')
    return today in date_dataframes


In [16]:
def fetch_dataframe():
    # Call the function and save the processed data to a CSV file
    df_processed = Covered_calls.get_processed_dataframe()
    today = pd.Timestamp.today().strftime('%Y-%m-%d')
    filename = f'Covered_calls_output_{today}.csv'
    df_processed.to_csv(filename, index=False)
    return df_processed

In [17]:
# Define a function to save a dataframe to a file with the given date as the filename
def save_dataframe(df, date):
    df.to_csv(f"{date}.csv", index=True)

In [18]:
# Define a function to load a dataframe from a file with the given date as the filename
def load_dataframe(date):
    df = pd.read_csv(f"{date}.csv", index_col=0)
    return df

In [19]:
# Define a function to update the dataframes in parallel dimensions
def update_dataframes(date_dataframes):
    today = time.strftime('%Y-%m-%d')
    start_date = datetime.datetime(2023, 2, 14).strftime('%Y-%m-%d')
    dates = pd.date_range(start=start_date, end=today, freq='D').strftime('%Y-%m-%d')

    # Check if df_processed has been fetched for today
    if is_dataframe_fetched_today(date_dataframes):
        df_today = date_dataframes[today]
        df_today = df_today.set_index('contractSymbol')
    else:
        df_today = fetch_dataframe()
        date_dataframes[today] = df_today
        save_dataframe(df_today, today)

    # Update the dataframes for all previous dates
    for date in dates:
        if date == today:
            continue

        if date not in date_dataframes:
            date_dataframes[date] = pd.DataFrame(index=pd.Index([], dtype='object'), columns=df_today.columns)

        df = date_dataframes[date]

        # Merge the new data with the existing data
        df_merged = pd.merge(df, df_today, on='contractSymbol', how='outer', suffixes=('', '_new'))
        df_merged = df_merged.drop(columns=[col for col in df_today.columns if col != 'contractSymbol'])
        df_merged = df_merged.rename(columns={col: col.replace('_new', '') for col in df_merged.columns})

        # Save the merged data to the file
        date_dataframes[date] = df_merged
        save_dataframe(df_merged, date)

    # Return the current day dataframe
    return df_today

In [20]:
# Create an empty dictionary to store the dataframes
date_dataframes = {}

# Update the dataframes and get the current day dataframe
df_today = update_dataframes(date_dataframes)

NameError: name 'covered_calls' is not defined

In [ ]:
# Define a function to export a dataframe to an Excel file
def export_dataframe(date_dataframes, date):
    df = date_dataframes[date]
    writer = pd.ExcelWriter(f"{date}.xlsx")
    df.to_excel(writer, sheet_name='data', index=True)
    writer.save()

In [ ]:
# Export a dataframe to an Excel file
export_dataframe(date_dataframes, '2023-02-14')